# Saving logits of original and adversarially attacked inputs

The "hack" below allows absolute path imports as if the notebook was a py-file run with `python -m abs.path.to.file.filename`

In [1]:
import sys
root_dir = !cd .. &&pwd
root_dir = root_dir[0] + "/"
sys.path.insert(0, root_dir)

In [2]:
print(root_dir)

/Users/philip/DIKU/bachelor_project/


### Imports

In [3]:
import torch
from models.classifier32 import classifier32
from our_modules.tin_tools import get_avg_osr_auroc_across_splits

### Setting the Device

In [4]:
if sys.platform == 'darwin':
    device = torch.device("mps")
else:
    gpu = torch.cuda.is_available()
    device = torch.device("cuda:0" if gpu else "cpu")

print("device:", device)

device: mps


## tiny-imagenet

In [5]:
path_to_pretrained_weights_folder = root_dir + "pretrained_weights/"
tin_val_root_dir = root_dir + "datasets/tiny-imagenet-200/val/images/"

### Saving plain logits

In [6]:
get_avg_osr_auroc_across_splits(path_to_pretrained_weights_folder, tin_val_root_dir, device, logdir=root_dir + "logits/tinyimagenet/")

100%|██████████| 5/5 [00:27<00:00,  5.57s/it]


0.8295037888888889

In [13]:
torch.load(root_dir + "logits/tinyimagenet/plain/logits_split_0.pt").shape

torch.Size([10000, 20])